In [11]:
import pandas as pd
import os
import gzip
import pickle
import openai
import torch.nn.functional as F
import torch

In [12]:
df_hardware_name = pd.read_parquet('datasets/extracted/hardware_name.parquet')
# remove rows where hardware_name is "Nothing"
df_hardware_name = df_hardware_name[df_hardware_name['hardware_name'] != 'Nothing']

# sort by date
df_hardware_name = df_hardware_name.sort_values(by=['date'])
df_hardware_name

In [15]:
df_hardware_price = pd.read_parquet('datasets/extracted/hardware_price.parquet')
# remove rows where hardware_price is "Nothing"
df_hardware_price = df_hardware_price[df_hardware_price['hardware_price'] != 'Nothing']

# sort by date
df_hardware_price = df_hardware_price.sort_values(by=['date'])
df_hardware_price

,date,hardware_price
6,2012-04-05,$150
7,2014-01-05,$810
0,2014-07-15,$850
10,2015-04-29,1.5 btc
1,2020-03-31,3605$
5,2021-09-26,<hardware price> 500EUR
